# Title

Импорт библиотек

In [198]:
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.model_selection import GridSearchCV


Загрузка данных

In [29]:
train_df = pd.read_csv('D:/Kode/MO/train.csv')
test_df = pd.read_csv('D:/Kode/MO/test.csv')

In [30]:
train_df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


Преобразование элементов столбца 'datetime' в формат datetime

In [31]:
test_df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


In [32]:
train_df['datetime'] = pd.to_datetime(train_df['datetime'])
test_df['datetime'] = pd.to_datetime(test_df['datetime'])

In [33]:
train_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    10886 non-null  datetime64[ns]
 1   season      10886 non-null  int64         
 2   holiday     10886 non-null  int64         
 3   workingday  10886 non-null  int64         
 4   weather     10886 non-null  int64         
 5   temp        10886 non-null  float64       
 6   atemp       10886 non-null  float64       
 7   humidity    10886 non-null  int64         
 8   windspeed   10886 non-null  float64       
 9   casual      10886 non-null  int64         
 10  registered  10886 non-null  int64         
 11  count       10886 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(8)
memory usage: 1020.7 KB


Вычленение из столбца 'datetime' месяц в столбец 'month', час в 'hour' и день недели в 'name_day'

In [34]:
train_df['month'] = train_df['datetime'].apply(lambda t: t.month)
train_df['hour'] = train_df['datetime'].apply(lambda t: t.hour)
train_df['name_day'] = train_df['datetime'].apply(lambda t: t.dayofweek)

test_df['month'] = test_df['datetime'].apply(lambda t: t.month)
test_df['hour'] = test_df['datetime'].apply(lambda t: t.hour)
test_df['name_day'] = test_df['datetime'].apply(lambda t: t.dayofweek)

Перетасуем столбцы так, чтоб столбцы, производные от 'datetime', находились рядом для красоты

In [35]:
train_df = train_df[['datetime', 'month', 'hour', 'name_day', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count']]
test_df = test_df[['datetime', 'month', 'hour', 'name_day', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed']]

In [36]:
train_df.head()

,datetime,month,hour,name_day,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,5,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,1,5,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,2,5,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,3,5,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,4,5,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [37]:
train_df.describe()

,datetime,month,hour,name_day,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
count,10886,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.00000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000
mean,2011-12-27 05:56:22.399411968,6.521495,11.541613,3.013963,2.506614,0.028569,0.680875,1.418427,20.23086,23.655084,61.886460,12.799395,36.021955,155.552177,191.574132
min,2011-01-01 00:00:00,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.82000,0.760000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2011-07-02 07:15:00,4.000000,6.000000,1.000000,2.000000,0.000000,0.000000,1.000000,13.94000,16.665000,47.000000,7.001500,4.000000,36.000000,42.000000
50%,2012-01-01 20:30:00,7.000000,12.000000,3.000000,3.000000,0.000000,1.000000,1.000000,20.50000,24.240000,62.000000,12.998000,17.000000,118.000000,145.000000
75%,2012-07-01 12:45:00,10.000000,18.000000,5.000000,4.000000,0.000000,1.000000,2.000000,26.24000,31.060000,77.000000,16.997900,49.000000,222.000000,284.000000
max,2012-12-19 23:00:00,12.000000,23.000000,6.000000,4.000000,1.000000,1.000000,4.000000,41.00000,45.455000,100.000000,56.996900,367.000000,886.000000,977.000000
std,NaN,3.444373,6.915838,2.004585,1.116174,0.166599,0.466159,0.633839,7.79159,8.474601,19.245033,8.164537,49.960477,151.039033,181.144454


# Дерево решений

Вынос целевой переменной в отдельный DataFrame

In [66]:
y_train = train_df['count']

In [69]:
y_train.head()

0    16
1    40
2    32
3    13
4     1
Name: count, dtype: int64

Удаление из train_df casual и registered, т.к. они не несут практической ценности и их нет в тестовых данных

In [38]:
train_df_no_cas_reg = train_df.drop(['datetime', 'casual','registered'], axis=1)
test_df_no_cas_reg =  test_df.drop(['datetime'], axis=1)

In [39]:
train_df_no_cas_reg.head()

,month,hour,name_day,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count
0,1,0,5,1,0,0,1,9.84,14.395,81,0.0,16
1,1,1,5,1,0,0,1,9.02,13.635,80,0.0,40
2,1,2,5,1,0,0,1,9.02,13.635,80,0.0,32
3,1,3,5,1,0,0,1,9.84,14.395,75,0.0,13
4,1,4,5,1,0,0,1,9.84,14.395,75,0.0,1


In [40]:
test_df_no_cas_reg.head()

,month,hour,name_day,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,1,0,3,1,0,1,1,10.66,11.365,56,26.0027
1,1,1,3,1,0,1,1,10.66,13.635,56,0.0000
2,1,2,3,1,0,1,1,10.66,13.635,56,0.0000
3,1,3,3,1,0,1,1,10.66,12.880,56,11.0014
4,1,4,3,1,0,1,1,10.66,12.880,56,11.0014


Кодирование категориальных признаков и добавление их в DataFrame

In [57]:
train_df_dummies = pd.concat([pd.get_dummies(train_df['name_day'], prefix="name_day"),
                              pd.get_dummies(train_df['season'], prefix="season"),
                              pd.get_dummies(train_df['weather'], prefix="weather"),
                              pd.get_dummies(train_df['holiday'], prefix="holiday"),
                              pd.get_dummies(train_df['workingday'], prefix="workingday"),
                              pd.get_dummies(train_df['workingday'], prefix="workingday")],
                              axis=1)

Добавление оставшихся признаков

In [58]:
train_df_dummies = pd.concat([train_df_dummies,
                              train_df['month'],
                              train_df['hour'],
                              train_df['temp'],
                              train_df['atemp'],
                              train_df['humidity'],
                              train_df['windspeed']],
                              axis=1)

In [60]:
train_df_dummies.head()

,name_day_0,name_day_1,name_day_2,name_day_3,name_day_4,name_day_5,name_day_6,season_1,season_2,season_3,...,workingday_0,workingday_1,workingday_0,workingday_1,month,hour,temp,atemp,humidity,windspeed
0,False,False,False,False,False,True,False,True,False,False,...,True,False,True,False,1,0,9.84,14.395,81,0.0
1,False,False,False,False,False,True,False,True,False,False,...,True,False,True,False,1,1,9.02,13.635,80,0.0
2,False,False,False,False,False,True,False,True,False,False,...,True,False,True,False,1,2,9.02,13.635,80,0.0
3,False,False,False,False,False,True,False,True,False,False,...,True,False,True,False,1,3,9.84,14.395,75,0.0
4,False,False,False,False,False,True,False,True,False,False,...,True,False,True,False,1,4,9.84,14.395,75,0.0


In [74]:
train_df_dummies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 27 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name_day_0    10886 non-null  bool   
 1   name_day_1    10886 non-null  bool   
 2   name_day_2    10886 non-null  bool   
 3   name_day_3    10886 non-null  bool   
 4   name_day_4    10886 non-null  bool   
 5   name_day_5    10886 non-null  bool   
 6   name_day_6    10886 non-null  bool   
 7   season_1      10886 non-null  bool   
 8   season_2      10886 non-null  bool   
 9   season_3      10886 non-null  bool   
 10  season_4      10886 non-null  bool   
 11  weather_1     10886 non-null  bool   
 12  weather_2     10886 non-null  bool   
 13  weather_3     10886 non-null  bool   
 14  weather_4     10886 non-null  bool   
 15  holiday_0     10886 non-null  bool   
 16  holiday_1     10886 non-null  bool   
 17  workingday_0  10886 non-null  bool   
 18  workingday_1  10886 non-nu

Функция для визуализации дерева решений

In [204]:
def get_tree_image(tree, x_train):
    plt.figure(figsize=((30,10)))
    plot_tree(tree,
            filled=True,
            feature_names=x_train.columns)
    plt.show()

Функция для оценки качества модели дерева решений

In [213]:
def score_d_tree(X, y, depth = 2, leaf=None, ratio=0.9, seed=17):
    
    x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=ratio, random_state=seed)

    d_tree = DecisionTreeRegressor(max_depth=depth, min_samples_leaf=leaf, random_state=seed)
    d_tree.fit(x_train, y_train)

    #pred = d_tree.predict(x_test)

    scored = d_tree.score(x_test, y_test)
    
    return scored

In [214]:
score_d_tree(train_df_dummies, y_train, depth=12, leaf=18)

0.8345344076821993

Нахождение лучших параметров дерева

In [311]:
# tree params for grid search
tree_params = {
               'max_depth': list(range(1, 20)), 
               'min_samples_leaf': list(range(1, 20))
              }

d_tree = DecisionTreeRegressor(max_depth=2, random_state=17)

tree_grid_1 = GridSearchCV(d_tree, tree_params,
                         cv=5, n_jobs=-1,
                        verbose=True)
tree_grid_1.fit(train_df_dummies, y_train)

Fitting 5 folds for each of 361 candidates, totalling 1805 fits


GridSearchCV(cv=5,
             estimator=DecisionTreeRegressor(max_depth=2, random_state=17),
             n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15, 16, 17, 18, 19],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                              12, 13, 14, 15, 16, 17, 18, 19]},
             verbose=True)

In [312]:
scored_1 = tree_grid_1.score(train_df_dummies, y_train)
print(scored_1)
print(tree_grid_1.best_params_)

0.8271040773387346
{'max_depth': 9, 'min_samples_leaf': 18}


In [314]:
# TODO: Настройка прараметров лишь ухудшает результат.
#       попроборать разделить train_df_dummies на доли
#       как в 4 лабе для оценивания на новых данных
#       и мб залить predict test_df на Kaggle